<a href="https://colab.research.google.com/github/oda251/mojiokoshi_tools/blob/main/Whisper_%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **使用方法**
#### 1. 上部メニューの「ランタイム」「ランタイムのタイプを変更」から、「T4 GPU」を選択してください
#### 2. 上から順番に、指示に従って入力等を行ってください
#### 3. 最後に、上部メニューの「ランタイム」から「すべてのセルを実行」をクリックすると、output.txtに結果が格納されます。

In [ ]:
#@title 必要なライブラリのインストール
!pip install -U openai-whisper
!sudo apt update && sudo apt install ffmpeg
!pip install pydub

import subprocess
import whisper
from pydub import AudioSegment
import os
from google.colab import files

In [7]:
#@title Whisperモデルを選択
#@markdown 1. モデルを選択してください。大きいほど精度と処理時間が大きくなります
モデル = "large" #@param ["large", "medium", "small", "base", "tiny"]
model = whisper.load_model(モデル)

100%|█████████████████████████████████████| 2.88G/2.88G [00:48<00:00, 64.1MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [9]:
#@title ファイルのアップロード
#@markdown - 音声ファイルをアップロードしてください
uploaded = files.upload()
audio_path = list(uploaded)[0]

if os.path.isfile(audio_path):
  print("アップロードが完了しました")
else:
  print("入力されたパスにファイルが見つかりませんでした")

Saving レコーディング (18).m4a to レコーディング (18).m4a
アップロードが完了しました


In [10]:
#@title 実行
#@markdown 1. 上部メニューの「ランタイム」から「すべてのセルを実行」をクリック
if not os.path.isfile(audio_path):
  print("ファイルが正しくアップロードされていません")

def convert_m4a_to_mp3(file_path, output_path):
    # ffmpegを使用してm4aをmp3に変換
    subprocess.run(['ffmpeg', '-i', file_path, output_path])

def transcribe_audio(file_path):
    # ここでは、modelは既にロードされていると想定
    global model
    sound = AudioSegment.from_file(file_path)
    duration = sound.duration_seconds
    result = []
    for i, j in enumerate(range(0, int(duration), 25)):
        left = j
        if left > 0:
          left += 3
        right = (j + 25)
        left *= 1000
        right *= 1000
        tgt = "tmp.mp3"
        sound[left:right].export(tgt, format="mp3")
        result.append(model.transcribe(tgt)["text"])
    return result

transcription_result = transcribe_audio(audio_path)

# 保存先の指定
output_path = '/content/'

# ディレクトリが存在しない場合は作成
if not os.path.exists(output_path):
    os.makedirs(output_path)

# 保存
export_file = output_path + "output.txt"
with open(export_file, 'w') as f:
    for txt in transcription_result:
      f.write(txt)
      f.write('\n')

print(f"ファイルが保存されました: {export_file}")

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


ファイルが保存されました: /content/output.txt
